## Data: Online Retail II

This Online Retail II data set contains all the transactions occurring for a UK-based and registered, non-store online retail between 01/12/2009 and 09/12/2011.The company mainly sells unique all-occasion gift-ware. Many customers of the company are wholesalers.

The dataset can be accessed at the following link: https://www.kaggle.com/datasets/mashlyn/online-retail-ii-uci/

Attribute Information:

- InvoiceNo: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.
- StockCode: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.
- Description: Product (item) name. Nominal.
- Quantity: The quantities of each product (item) per transaction. Numeric.
- InvoiceDate: Invice date and time. Numeric. The day and time when a transaction was generated.
- UnitPrice: Unit price. Numeric. Product price per unit in sterling (Â£).
- CustomerID: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.
- Country: Country name. Nominal. The name of the country where a customer resides.

## Environment Setup

In [ ]:
import os

import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from datetime import datetime
from dateutil import relativedelta

from collections import Counter

pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.max_rows', 10)

In [ ]:
import warnings

# Suppress FutureWarning from plotly
warnings.filterwarnings("ignore", category=FutureWarning, module="_plotly_utils")
warnings.filterwarnings("ignore", category=FutureWarning, module="plotly")

In [ ]:
from Cohort.Cohort import Cohort
from Cohort.CohortDate import CohortDate
from Cohort.CohortPeriod import CohortPeriod
from Cohort.AnalyzeCohortDate import AnalyzeCohortDate

## Data Preparation

In [ ]:
def get_uk_online_retail_2():
    global COL_CUSTOMER_ID, COL_ORDER_DATE, COL_ORDER_ID, COL_VALUE, COL_QUANTITY
    COL_CUSTOMER_ID = 'Customer ID'
    COL_ORDER_DATE = 'InvoiceDate'
    COL_ORDER_ID = 'Invoice'
    COL_VALUE = 'Sales'
    COL_QUANTITY = 'Quantity'
    
    # Load data
    pathname = os.path.join("F:\\Data\\datas", "online_retail_II.csv")
    df = pd.read_csv(
        pathname, 
        dtype = {'Customer ID': 'str', 'Invoice': str},
        parse_dates = ['InvoiceDate']
    )    
    
    # Drop missing values
    df = df.dropna()
    
    # Ignore duplicate rows
#     df.loc[df.duplicated()].shape
    
    # Drop the TEST stock code    
    df = df[~df['StockCode'].str.contains('TEST')]
    
    # Drop 0 price
    price_0 = df[df['Price'] == 0].index
    df = df.drop(index=price_0)
    
    # Drop cancellation invoice or quantity with <= 0
#     df = df[~df['Invoice'].str.startswith('C')]  
    df = df[~df['Quantity'] <= 0]
    
    # Get Sales feature
    df['Sales'] = df['Price'] * df['Quantity']
    
    return df

In [ ]:
df = get_uk_online_retail_2()

## Cohort Analysis

In [ ]:
MyCohort = Cohort(df, COL_CUSTOMER_ID, COL_ORDER_DATE, COL_ORDER_ID)

### Cohort Data

#### How our cohort's data look like (in long rows format)

- Cohort
    - A cohort is a group of customers who made their first purchase in the same period (e.g. month). 
    - This column indicates the month when the cohort was formed.
- Period Date
    - The specific month corresponding to the value of data. 
    - This indicates the month in which the value (e.g. Sales) were recorded.
- Sales
    - The specific value of data

In [ ]:
MyCohort.get_cohort_agg(period_date='Q', aggs={'Sales': ['sum', 'mean'], COL_CUSTOMER_ID: ['unique', 'nunique']})

In [ ]:
cohort_data = MyCohort.get_cohort_agg(col_value=COL_VALUE, period_date='Q', aggfunc='sum')
cohort_data['Period Date'] = cohort_data['Period Date'].dt.to_timestamp(freq='Q', how='S')

In [ ]:
cohort_data

In [ ]:
px.line(cohort_data, x='Period Date', y='Sales', color='Cohort', template='plotly_white', title = 'Sales Journey in each Cohort')

#### Get cohort feature

In [ ]:
MyCohort.get_members_cohort(period_date='M', period_date_step='M')

#### Get Summary Cohort

In [ ]:
MyCohort.get_cohort_summary('Sales')

In [ ]:
MyCohort.get_top_contributors('Sales')

### Cohort Date Table

In [ ]:
MyCohortDate = CohortDate(df, COL_CUSTOMER_ID, COL_ORDER_DATE, COL_ORDER_ID)

In [ ]:
MyCohortDate.get_table('Sales', period_date='Q')

In [ ]:
MyCohortDate.get_pivot('Sales', period_date='Q')

In [ ]:
myCohort = AnalyzeCohortDate(df, COL_CUSTOMER_ID, COL_ORDER_DATE, COL_ORDER_ID)

In [ ]:
myCohort.get_active_customers('M')

In [ ]:
myCohort.get_inactive_customers('Sales', latest_active_period=3, show_contributions=True)

###  Cohort Period Table

In [ ]:
MyCohortPeriod = CohortPeriod(df, COL_CUSTOMER_ID, COL_ORDER_DATE, COL_ORDER_ID)

In [ ]:
MyCohortPeriod.get_table(COL_VALUE, period_date='Q')

In [ ]:
MyCohortPeriod.get_pivot(COL_VALUE, period_date='Q', period_date_step='M')

## Report

In [ ]:
cohort_data_2009_12 = cohort_data[cohort_data['Cohort'] == '2009-12'].copy()
cohort_data_2009_12['Period Date'] = cohort_data_2009_12['Period Date'].dt.to_timestamp(freq='M', how='S')

### Cohort Compositions

### Retention Heatmap